In [30]:
import mne
import warnings
import pandas as pd
import numpy as np
from scipy.signal import welch
from processing_functions import ppt_id, create_epochs, create_numeric_labels, relative_band_power, absolute_band_power
warnings.simplefilter('ignore')

This notebook produces a numpy array according to the settings below that will then later be saved with the indicated filename. Feel free to change the settings and filename as needed. freq_bands is measured in hertz, while epoch_length is measured in 0.002 seconds = 2 ms (so for instance epoch_length = 2000 means 4 seconds). overlap_ratio specifies the percentage of overlap between successive epochs. nperseg specifies the size of the window used in the welch signal processing function, measured in the same units as epoch_length. 

In [8]:
epoch_length = 2000
overlap_ratio = 0.5
freq_bands = np.array([0.5,4.0,8.0,13.0,25.0,45.0])
nperseg = 256

The processing function is coded up below. The default values of sample_freq and total_subjects shouldn't need to be modified since these are built-in features of our data. This function first loads in the raw eeg data, converts it to a pandas dataframe, and then converts that to a numpy array. The epochs are then created from this array with the epoch length and overlap ratio specified above. These are then passed into the welch function for processing which produces the psd data with a frequency resolution determined by nperseg. This is then passed into the relative_band_power function to get our features. Finally the maximum number of epochs per participant is calculated and the participants with less than that number of epochs have their feature arrays zero-padded up to that maximum length. The resulting output array has dimension (total_subjects) x (max epoch length) x (len(freq_bands)-1) x (number of channels). That last number is always fixed at 19 for us since it's a characteristic of our data. Right now the number of bands is always 5, but that could potentially be adjusted in the future. We also return epoch_lengths, which is an array that we can use to easily exclude the zero-padded parts of the features array (we only pad the features array so that it can be saved as a numpy array while preserving its structure).

If you also want the absolute band power and total band power arrays you can set absolute = True. This will return two additional arrays, one of which is an array with the absolute band power in each frequency band and the other which gives the total band power in the frequency band. The units of these output arrays are uV^2/Hz (microvolts squared per hertz).

In [54]:
def process(epoch_length,overlap_ratio,freq_bands,nperseg, absolute=False, sample_freq=500,total_subjects=88):
    features = []
    if absolute:
        absolute_features = []
        total_features = []
    for i in range(total_subjects):
        ppt = i + 1
        raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                    + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
        export = raw_data.to_data_frame()
        ppt_array = export.iloc[:,range(1,len(export.columns))].values
        ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
        freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq,nperseg=nperseg, axis=1)
        ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
        features += [ppt_rbp]
        if absolute:
            ppt_abp = absolute_band_power(ppt_psd,freqs,freq_bands)
            ppt_tbp = absolute_band_power(ppt_psd,freqs,[freq_bands[0],freq_bands[-1]])
            absolute_features += [ppt_abp]
            total_features += [ppt_tbp]
    num_epochs = np.array([feature.shape[0] for feature in features])
    max_epoch_length = np.max(num_epochs)
    for i in range(total_subjects):
        shape = list(features[i].shape)
        shape[0] = max_epoch_length-shape[0]
        features[i] = np.concatenate((features[i],np.zeros(shape)),axis=0)
        if absolute:
            absolute_features[i] = np.concatenate((absolute_features[i],np.zeros(shape)),axis=0)
            total_shape = list(total_features[i].shape)
            total_shape[0] = max_epoch_length-total_shape[0]
            total_features[i] = np.concatenate((total_features[i],np.zeros(total_shape)),axis=0)
    if not absolute:
        return num_epochs, np.array(features)
    else:
        return num_epochs, np.array(features), np.array(absolute_features), np.array(total_features)
    

For processing and saving you can specify a list of filenames below and then run the process_and_save function. Two filenames should be specified if absolute=False (default) and four should be specified if absolute=True. 

In [1]:
filenames = ['num_epochs','rbp_features']
abs_filenames = ['num_epochs','rbp_features','abp_features','tbp_features']

In [52]:
def process_and_save(epoch_length,overlap_ratio,freq_bands,nperseg, filenames, absolute=False, sample_freq=500,total_subjects=88):
    if not absolute:
        num_epochs, rbp_features = process(epoch_length,overlap_ratio,freq_bands,nperseg, absolute=absolute, sample_freq=sample_freq,total_subjects=total_subjects)
        np.save(filenames[0],num_epochs)
        np.save(filenames[1],rbp_features)
    else:
        num_epochs, rbp_features, abp_features, tbp_features = process(epoch_length,overlap_ratio,freq_bands,nperseg, absolute=absolute, sample_freq=sample_freq,total_subjects=total_subjects)
        np.save(filenames[0],num_epochs)
        np.save(filenames[1],rbp_features)
        np.save(filenames[2],abp_features)
        np.save(filenames[3],tbp_features)

Example for absolute=False.

In [53]:
process_and_save(epoch_length,overlap_ratio,freq_bands,nperseg,filenames, absolute=False)

And for absolute=True.

In [ ]:
process_and_save(epoch_length,overlap_ratio,freq_bands,nperseg,abs_filenames, absolute=True)